In [86]:
import re
from langchain.document_loaders import PyPDFLoader

import json
import csv
import requests
import time
import os


def get_data_in_between_the_numbers(file_path ):
    """
    Extracts the text data between consecutive question numbers in a given PDF.

    :param file_path: Path to the PDF file.
    :param questions_to_fetch: A list of question numbers (e.g., ["1.1", "1.2", "1.3"]).
    :return: A dictionary mapping each question number to its associated text.
    """
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    num_pages = len(pages)  # Get total number of pages

    extracted_data = {}  # Store question numbers with corresponding extracted text
    all_text = []  # Store all text from the document for processing

    # Combine text from all pages while ignoring the first line on each page
    for page_number in range(num_pages):
        print(f"Processing page {page_number + 1}...")  # Current page (1-based index)

        current_lines = pages[page_number].page_content.strip().split("\n")

        # Remove the first line from all pages except the first one
        if page_number == 0:
            page_text = "\n".join(current_lines)
        else:
            page_text = "\n".join(current_lines[1:]) if len(current_lines) > 1 else ""

        all_text.append(page_text)

    combined_text = "\n".join(all_text)  # Merge text across all pages
    #print(combined_text)
    return combined_text

def generate_decimal_list(number,start=1, end=63):
    """
    Generates a list of decimal numbers in the format 1.1 to 1.N, ensuring each number appears at the start of lines.

    :param start: The starting integer (always 1 in this case).
    :param end: The ending integer that defines the second digit range.
    :return: A list of formatted decimal numbers as strings.
    """
    return [f"{number}.{i}" for i in range(start, end + 1)]

def remove_phrases(text, phrases):
    """
    Remove specific phrases from the given text.

    :param text: The input text.
    :param phrases: A list of phrases to remove.
    :return: Cleaned text with the phrases removed.
    """
    for phrase in phrases:
        text = re.sub(r'\b' + re.escape(phrase) + r'\b', '', text)  # Remove exact phrase
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

def extract_text_between_sections(text, start_section, end_section):
    """
    Extracts text between two numbered sections (e.g., 1.1 and 1.2).

    :param text: The full text from which to extract.
    :param start_section: The starting section number (e.g., '1.1').
    :param end_section: The ending section number (e.g., '1.2').
    :return: Extracted text between the sections, or an empty string if not found.
    """
    pattern = rf"{re.escape(start_section)}\s*(.*?)\s*{re.escape(end_section)}"
    match = re.search(pattern, text, re.DOTALL)

    return match.group(1).strip() if match else ""

def open_ai_resposne(api_key,api_url,ocr_extracted_text):

  # Updated prompt to format output like the second image
  improved_prompt = f"""
  You are a **physics problem-solving expert** with deep knowledge of **physics concepts, equations, and problem-solving techniques**. Your task is to:

  1. **Correct any errors in the given content**, ensuring:
    - Mathematical symbols are represented correctly (e.g., fractions → ½, exponents → ², ³, square roots → √, Greek letters → π, ω).
    - Equation formatting is accurate (e.g., dx/dt, ∑, √, π, ÷, ×).
    - Any missing or misinterpreted characters are restored.

  2. **Format the response as a structured and readable physics problem**, ensuring:
    - Proper indentation and spacing.
    - No additional explanations beyond solving the problem.
    - Clear separation between given equations and solutions.

  3. **Return the response in plain text, preserving all mathematical symbols exactly as they should appear.**
    - **Do NOT return LaTeX.**
    - **Do NOT use code block formatting (` ``` `).**
    - **Use real Unicode characters for exponents, fractions, and mathematical notations.**

  Now, process the following physics content and provide a complete, well-structured response:
  ---
  {ocr_extracted_text}
  ---
  """



  # Conversation history with system instructions and user prompt
  conversation_history = [
      {'role': 'system', 'content': "You are a physics expert that corrects OCR-extracted physics problems and formats them exactly like the original."},
      {'role': 'user', 'content': improved_prompt}
  ]

  # Set up the request headers
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json'
  }

  # Set up the request payload
  data = {
      'model': 'gpt-4o',  # Use GPT-4o for better reasoning
      'messages': conversation_history,
      'max_tokens': 1000,  # Increase tokens for detailed responses
      'temperature': 0.3  # Lower temperature for factual accuracy
  }

  # Make the POST request to the OpenAI API
  response = requests.post(api_url, headers=headers, json=data, timeout=30)

  # Print the response from the server
  if response.status_code == 200:
      print('API key is working!')
      print('Response:\n')
      #print(response.json()['choices'][0]['message']['content'])
      return response.json()['choices'][0]['message']['content']
  else:
      print('Failed to authenticate with the API key.')
      print('Status Code:', response.status_code)
      #print('Response:', response.text)
      return response.text


def open_ai_response_with_retries(api_key, api_url, extracted_text, retries=6, delay=20):

    # Updated prompt to format output into LaTeX
    improved_prompt = f"""
    You are an expert in LaTeX formatting for the physics problems and solutions. Your task is to convert the given physics text into properly formatted LaTeX.

    {extracted_text}

    """

    # Updated conversation history for LaTeX conversion
    conversation_history = [
        {'role': 'system', 'content': "You are a LaTeX expert specializing in physics equations. Convert the input text into fully formatted LaTeX code, ensuring correct syntax, structure, and alignment."},
        {'role': 'user', 'content': improved_prompt}
    ]

    # Set up the request headers
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }

    # Set up the request payload
    data = {
        'model': 'gpt-4o',  # Use GPT-4o for better reasoning
        'messages': conversation_history,
        'max_tokens': 1000,  # Increase tokens for detailed responses
        'temperature': 0.3  # Lower temperature for factual accuracy
    }

    for attempt in range(retries):
        try:
            response = requests.post(api_url, headers=headers, json=data, timeout=120)
            if response.status_code == 200:
                return response.json().get('choices', [{}])[0].get('message', {}).get('content', '')
            else:
                print(f"Attempt {attempt+1} failed: {response.text}")
        except requests.exceptions.Timeout:
            delay = attempt*delay
            print(f"Timeout error on attempt {attempt+1}, retrying in {delay} seconds...")
            time.sleep(delay)

    return "Error: Unable to process request after multiple attempts."


def keep_text_after_phrase(text, phrase):
    """
    Finds the first occurrence of a phrase in the text and keeps everything after it.

    :param text: The input text.
    :param phrase: The phrase after which text should be kept.
    :return: The cleaned text with everything after the specified phrase.
    """
    match = re.search(re.escape(phrase), text, re.IGNORECASE)  # Match exact phrase
    return text[match.end():].strip() if match else ""  # Keep everything after the phrase

def find_and_slice_text_from_section(text, section_number):
    """
    Finds the starting index of a given section number in the text and slices the text from that index.

    :param text: The full text where the section appears.
    :param section_number: The section number to find (e.g., "1.1").
    :return: The sliced text starting from the section number, or an empty string if not found.
    """
    #match = re.search(rf'\b{re.escape(section_number)}\b', text)  # Match the section exactly
    pattern = rf'\b{re.escape(section_number)}(?!\d)' 
    match = re.search(pattern, text)
    if match:
        index = match.start()  # Get index of section
        print("index: ",index)
        print("text after index: ",text[index:].strip())
        data  = text[index:].strip()
        return section_number,data  # Slice text from that index
    else:
        print("couldn't find the start number: ",section_number)
        section_number = float(section_number)+0.1
        section_number = str(section_number)
        print("New start number: ",section_number)
        return find_and_slice_text_from_section(text, section_number)
        #return ""  # Return empty string if not found






#Main
def main(file_path,phrases_to_remove,strat_number,question_end,question_start = 1):
  combined_text = get_data_in_between_the_numbers(file_path)


  cleaned_combined_text = remove_phrases(combined_text, phrases_to_remove)
  #print("cleaned_combined_text: ",cleaned_combined_text)
  data = find_and_slice_text_from_section(cleaned_combined_text,strat_number)
  strat_number = data[0]
  cleaned_combined_text = data[1]
  #print("cleaned_combined_text: ",cleaned_combined_text)
  print("strat_number:",strat_number)

  #print("cleaned_combined_text: ",cleaned_combined_text)
  # Generate the list of questions
#   print("strat_number.split('.')[0]",strat_number.split('.')[0])
#   print("strat_number.split('.')[0]",strat_number.split('.')[1])
  questions_to_fetch = generate_decimal_list(number = strat_number.split('.')[0],start=int(strat_number.split('.')[1]), end=question_end)
  #print("questions_to_fetch:", questions_to_fetch)

  cleaned_combined_text = cleaned_combined_text + "\n"+ questions_to_fetch[-1]
  # Your API Key and API URL
  api_key = 'He4E9kjuR31z3NBjiE8Hjzir66Fp7KyBjCWa9qPDBXVntjUw'
  api_url ='https://fauengtrussed.fau.edu/provider/generic/chat/completions'
  # CSV File Setup
  # Convert to CSV filename
  # Create "csv" folder if it doesn't exist
  csv_folder = "csv"
  os.makedirs(csv_folder, exist_ok=True)  # Ensures the folder exists

  # Convert PDF filename to CSV and place it in the "csv" folder
  csv_filename = os.path.join(csv_folder, os.path.splitext(os.path.basename(file_path))[0] + ".csv")
  #csv_filename = os.path.splitext(os.path.basename(file_path))[0] + ".csv"
  with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file)
      writer.writerow(["Question_number", "Solutions"])  # Writing headers
      for i in range(len(questions_to_fetch) - 1):  # Ensures the last item isn't used as start
          start = questions_to_fetch[i]
          end = questions_to_fetch[i + 1]
          print(f"Processing section: {start} to {end}")
          extracted_text = extract_text_between_sections(cleaned_combined_text, start, end)
          print("extracted_text",extracted_text)
          cleaned_combined_text = keep_text_after_phrase(cleaned_combined_text,extracted_text)
          #print("cleaned_combined_text: ",cleaned_combined_text)
          time.sleep(50)
          open_ai_response = open_ai_response_with_retries(api_key,api_url,extracted_text)

          print(f"OpenAI Response from {start} to {end}:")
          print(open_ai_response)
          # Write to CSV
          writer.writerow([start, open_ai_response])





In [87]:
# solutions_file_path = ["content/Kinematics_and_statics_solutions.pdf","content/Particle_Dynamics_solutions.pdf","content/Rotational_Kinematics_solutions.pdf",
#                       "content/Rotational_Dynamics_solutions.pdf","content/Gravitation_solutions.pdf","content/Oscillations_solutions.pdf",
#                       "content/Lagrangian_and_Hamiltonian_Mechanics_solutions.pdf","content/Waves_solutions.pdf","content/FluidDynamics_solutions.pdf",
#                       "content/HeatandMatter_solutions.pdf","content/Electrostatics_solutions.pdf","content/ElectricCircuits_solutions.pdf",
#                       "content/Electromagnetism_I_solutions.pdf","content/Electromagnetism_II_solutions.pdf","content/Optics_solutions.pdf"]


# Kinematics_phrases_to_remove = ["1 Kinematics and Statics", "1.3 Solutions","1.2.1 Motion in One Dimension","1.3.1 Motion in One Dimension","1.2.2 Motion in Resisting Medium","1.3.2 Motion in Resisting Medium","1.2.3 Motion in Two Dimensions","1.3.3 Motion in Two Dimensions","1.2.4 Force and Torque","1.3.4 Force and Torque","1.2.5 Centre of Mass","1.3.5 Centre of Mass","1.2.6 Equilibrium","1.3.6 Equilibrium"]
# Particle_Dynamics_to_remove = ["2.2 Problems","2.2.1 Motion of Blocks on a Plane","2.2.2 Motion on Incline","2.2.3 Work, Power, Energy","2.2.4 Collisions","2.2.5 Variable Mass","2.3 Solutions","2.3.1 Motion of Blocks on a Plane","2.3.2 Motion on Incline","2.3 Solutions","2.3.3 Work, Power, Energy","2.3.4 Collisions","2.3.5 Variable Mass","2 Particle Dynamics"]
# RotationalKinematics_to_remove = ["3.2 Problems","3.2.1 Motion in a Horizontal Plane","3.2.2 Motion in a Vertical Plane","3.2.3 Loop-the-Loop","3.3 Solutions","3.3.1 Motion in a Horizontal Plane","3 Rotational Kinematics","3.3.2 Motion in a Vertical Plane","3.3.3 Loop-the-Loop"]
# RotationalDynamics_to_remove = ["4 Rotational Dynamics","4.2 Problems","4.2.1 Moment of Inertia","4.2.2 Rotational Motion","4.2.3 Coriolis Acceleration","4.3 Solutions","4.3.1 Moment of Inertia","4.3.2 Rotational Motion","4.3.3 Coriolis Acceleration"]
# Gravitation_to_remove = ["5.2 Problems","5.2.1 Field and Potential","5 Gravitation","5.2.2 Rockets and Satellites","5.3 Solutions","5.3.1 Field and Potential","5.3.2 Rockets and Satellites","5.3 Solutions"]
# Oscillations_to_remove = ["6.2 Problems","6.2.1 Simple Harmonic Motion (SHM)","6 Oscillations","6.2.2 Physical Pendulums","6.2.3 Coupled Systems of Masses and Springs","6.2.4 Damped Vibrations","6.3 Solutions","6.3.1 Simple Harmonic Motion (SHM)","6.3.2 Physical Pendulums","6.3.3 Coupled Systems of Masses and Springs","6.3.4 Damped Vibrations"]
# LagrangianandHamiltonianMechanics_to_remove = ["7.2 Problems","Hamiltonian’s Canonical Equations","7.3 Solutions","7 Lagrangian and Hamiltonian Mechanics"]
# Waves_to_remove = ["8.2 Problems","8 Waves","8.3 Solutions","8.2.1 Vibrating Strings","8.3.1 Vibrating Strings","8.2.2 Waves in Solids","8.3.2 Waves in Solids","8.3.3 Waves in Liquids","8.2.4 Sound Waves","8.3.4 Sound Waves","8.2.5 Doppler Effect","8.3.5 Doppler Effect","8.2.6 Shock Wave","8.3.6 Shock Wave","8.2.7 Reverberation","8.3.7 Reverberation","8.2.8 Echo","8.3.8 Echo","8.2.9 Beat Frequency","8.3.9 Beat Frequency","8.2.10 Waves in Pipes","8.3.10 Waves in Pipes"]
# FluidDynamics_to_remove = ["9.2 Problems","9 Fluid Dynamics","9.3 Solutions","9.2.1 Bernoulli’s Equation","9.3.1 Bernoulli’s Equation","9.2.2 Torricelli’s Theorem","9.3.2 Torricelli’s Theorem","9.2.3 Viscosity","9.3.3 Viscosity"]
# HeatandMatter_to_remove = ["10.2 Problems","10 Heat and Matter","10.3 Solutions","10.2.1 Kinetic Theory of Gases","10.3.1 Kinetic Theory of Gases","10.2.2 Thermal Expansion","10.3.2 Thermal Expansion","10.2.3 Heat Transfer","10.3.3 Heat Transfer","10.2.4 Specific Heat and Latent Heat","10.3.4 Specific Heat and Latent Heat","10.2.5 Thermodynamics","10.3.5 Thermodynamics","10.2.6 Elasticity","10.3.6 Elasticity","10.2.7 Surface Tension","10.3.7 Surface Tension"]
# Electrostatic_to_remove = ["11.2 Problems","11 Electrostatic","11.3 Solutions","11.2.1 Electric Field and Potential","11.3.1 Electric Field and Potential","11.2.2 Gauss’ Law","11.3.2 Gauss’ Law","11.2.3 Capacitors","11.3.3 Capacitors"]
# Electric_Circuits_to_remove = ["12.2 Problems","12 Electric Circuits","12.3 Solutions","12.2.1 Resistance, EMF, Current, Power","12.3.1 Resistance, EMF, Current, Power","12.2.2 Cells","12.3.2 Cells","12.2.3 Instruments","12.3.3 Instruments","12.2.4 Kirchhoff’s Laws","12.3.4 Kirchhoff’s Laws"]
# Electromagnetism_I_to_remove = ["13.2 Problems","13 Electromagnetism I","13.3 Solutions","13.2.1 Motion of Charged Particles in Electric and Magnetic Fields","13.3.1 Motion of Charged Particles in Electric and Magnetic Fields","13.2.2 Magnetic Induction","13.3.2 Magnetic Induction","13.2.3 Magnetic Force","13.3.3 Magnetic Force","13.2.4 Magnetic Energy, Magnetic Dipole Moment","13.3.4 Magnetic Energy, Magnetic Dipole Moment","13.2.5 Faraday’s Law","13.3.5 Faraday’s Law","13.2.6 Hall Effect","13.3.6 Hall Effect"]
# Electromagnetism_II_to_remove = ["14.2 Problems","14 Electromagnetism II","14.3 Solutions","14.2.1 The RLC Circuits","14.3.1 The RLC Circuits","14.2.2 Maxwell’s Equations, Electromagnetic Waves,Poynting Vector","14.3.2 Maxwell’s Equations, Electromagnetic Waves,Poynting Vector","14.2.3 Phase Velocity and Group Velocity","14.3.3 Phase Velocity and Group Velocity","14.2.4 Waveguides","14.3.4 Waveguides"]
# Optics_to_remove = ["15.2 Problems","15 Optics","15.3 Solutions","15.2.1 Geometrical Optics","15.3.1 Geometrical Optics","15.2.2 Prisms and Lenses","15.3.2 Prisms and Lenses","15.2.3 Matrix Methods","15.3.3 Matrix Methods","15.2.4 Interference","15.3.4 Interference","15.2.5 Diffraction","15.3.5 Diffraction","15.2.6 Polarization","15.3.6 Polarization"]

# phrases_to_remove_list = [Kinematics_phrases_to_remove,Particle_Dynamics_to_remove,RotationalKinematics_to_remove,RotationalDynamics_to_remove,Gravitation_to_remove,Oscillations_to_remove,LagrangianandHamiltonianMechanics_to_remove,Waves_to_remove,FluidDynamics_to_remove,HeatandMatter_to_remove,Electrostatic_to_remove,Electric_Circuits_to_remove,Electromagnetism_I_to_remove,Electromagnetism_II_to_remove,Optics_to_remove]

# strats_numbers_list = ['1.1','2.1','3.1','4.1','5.1','6.1','7.1','8.1','9.1','10.1','11.1','12.1','13.1','14.1','15.1']
# question_end_list = [64,68,44,74,55,68,36,87,30,71,99,57,86,105,73]
# # question_start_list = [1,1]

# # Correcting the loop
# for i in range(len(phrases_to_remove_list)):  # Using phrases_to_remove_list
#     file_path = solutions_file_path[i]
#     phrases_to_remove = phrases_to_remove_list[i]  # Now properly defined
#     strat_number = strats_numbers_list[i]
#     #question_start = question_start_list[i]
#     #print("question_start: ", question_start)
#     question_end = question_end_list[i]
    
#     # Assuming 'main' is a function you have defined somewhere
#     main(file_path, phrases_to_remove, strat_number, question_end, question_start = 1 )


In [88]:
solutions_file_path = [
                      "content/Electromagnetism_I_solutions.pdf","content/Optics_solutions.pdf"]


# HeatandMatter_to_remove = ["10.2 Problems","10 Heat and Matter","10.3 Solutions","10.2.1 Kinetic Theory of Gases","10.3.1 Kinetic Theory of Gases","10.2.2 Thermal Expansion","10.3.2 Thermal Expansion","10.2.3 Heat Transfer","10.3.3 Heat Transfer","10.2.4 Specific Heat and Latent Heat","10.3.4 Specific Heat and Latent Heat","10.2.5 Thermodynamics","10.3.5 Thermodynamics","10.2.6 Elasticity","10.3.6 Elasticity","10.2.7 Surface Tension","10.3.7 Surface Tension"]
# Electric_Circuits_to_remove = ["12.2 Problems","12 Electric Circuits","12.3 Solutions","12.2.1 Resistance, EMF, Current, Power","12.3.1 Resistance, EMF, Current, Power","12.2.2 Cells","12.3.2 Cells","12.2.3 Instruments","12.3.3 Instruments","12.2.4 Kirchhoff’s Laws","12.3.4 Kirchhoff’s Laws"]
Electromagnetism_I_to_remove = ["13.2 Problems","13 Electromagnetism I","13.3 Solutions","13.2.1 Motion of Charged Particles in Electric and Magnetic Fields","13.3.1 Motion of Charged Particles in Electric and Magnetic Fields","13.2.2 Magnetic Induction","13.3.2 Magnetic Induction","13.2.3 Magnetic Force","13.3.3 Magnetic Force","13.2.4 Magnetic Energy, Magnetic Dipole Moment","13.3.4 Magnetic Energy, Magnetic Dipole Moment","13.2.5 Faraday’s Law","13.3.5 Faraday’s Law","13.2.6 Hall Effect","13.3.6 Hall Effect"]
Optics_to_remove = ["15.2 Problems","15 Optics","15.3 Solutions","15.2.1 Geometrical Optics","15.3.1 Geometrical Optics","15.2.2 Prisms and Lenses","15.3.2 Prisms and Lenses","15.2.3 Matrix Methods","15.3.3 Matrix Methods","15.2.4 Interference","15.3.4 Interference","15.2.5 Diffraction","15.3.5 Diffraction","15.2.6 Polarization","15.3.6 Polarization"]

phrases_to_remove_list = [Electromagnetism_I_to_remove,Optics_to_remove]

strats_numbers_list = ['13.1','15.1']
question_end_list = [86,73]
# question_start_list = [1,1]

# Correcting the loop
for i in range(len(phrases_to_remove_list)):  # Using phrases_to_remove_list
    file_path = solutions_file_path[i]
    phrases_to_remove = phrases_to_remove_list[i]  # Now properly defined
    strat_number = strats_numbers_list[i]
    #question_start = question_start_list[i]
    #print("question_start: ", question_start)
    question_end = question_end_list[i]
    
    # Assuming 'main' is a function you have defined somewhere
    main(file_path, phrases_to_remove, strat_number, question_end, question_start = 1 )


Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)


Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
couldn't find the start number:  13.1
New start number:  13.2
index:  1391
text after index:  13.2(a) Kp =1 2 q2r2B2 mp =1 2 ×(1.6×10−19)2(0.25)2(1.5)2 1.66×10−27 =2.17×10−13J=2.17×10−12 1.6×10−13 MeV=13.56 MeV fp = Bq 2πmp = 1.5×1.6×10−19 2π×1.66×10−27 =2.3×107Hz=23 MHz (b) Kα ≃1 2 (2e)2r2B2 4mp =Kp =13

Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)


OpenAI Response from 13.85 to 13.86:
```latex
\documentclass{article}
\usepackage{amsmath}

\begin{document}

If \( R_H \) is the Hall coefficient and \( \sigma \) is the electrical conductivity, then the mobility \( \mu \) is given by

\[
\mu = R_H \sigma = (-7.3 \times 10^{-5}) (2 \times 10^3) = -0.146 \, \text{m}^2/\text{V/s}
\]

The magnitude is \( 0.146 \, \text{m}^2/\text{V/s} \).

\end{document}
```
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Proce